In [9]:
import torch

from torchfem import Solid
from torchfem.io import export_mesh

# 3D Box with a single element

In [10]:
# Material model
E = 1000.0
nu = 0.3
lbd = (E * nu) / ((1.0 + nu) * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))

# E_xx, E_yy, E_zz, E_yz, E_xz, E_xy
C = torch.tensor(
    [
        [lbd + 2.0 * mu, lbd, lbd, 0.0, 0.0, 0],
        [lbd, lbd + 2.0 * mu, lbd, 0.0, 0.0, 0],
        [lbd, lbd, lbd + 2.0 * mu, 0.0, 0.0, 0],
        [0.0, 0.0, 0.0, mu, 0.0, 0],
        [0.0, 0.0, 0.0, 0.0, mu, 0],
        [0.0, 0.0, 0.0, 0.0, 0.0, mu],
    ]
)

nodes = torch.tensor(
    [[i, j, k] for k in [0.0, 1.0] for j in [0.0, 1.0] for i in [0.0, 1.0]]
)
elements = [[0, 1, 3, 2, 4, 5, 7, 6]]
forces = torch.zeros_like(nodes)
forces[nodes[:, 0] == 1.0, 0] = 100.0
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True

box = Solid(nodes, elements, forces, constraints, C)

u, f = box.solve()

export_mesh(box, "temp.vtk", nodal_data={"Disp": u})